In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import os
import sys
import tqdm
import time
import torch
import psutil
import warnings
import numpy as np
# from chnet.cahn_hill import ch_run_torch
from toolz.curried import pipe, curry, compose, memoize

In [3]:
def memReport():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            print(type(obj), obj.size())
    
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)
        
def get_free_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    return np.argmax(memory_available)

In [4]:
fft = curry(np.fft.fft)
ifft = curry(np.fft.ifft)
fftn = curry(np.fft.fftn)
ifftn = curry(np.fft.ifftn)
fftshift = curry(np.fft.fftshift)
ifftshift = curry(np.fft.ifftshift)
torch_rfft = curry(torch.rfft)
torch_irfft = curry(torch.irfft)

def conjugate(x):
    y = torch.empty_like(x)
    y[..., 1] = x[..., 1] * -1
    y[..., 0] = x[... , 0]
    return y

@curry
def mult(x1, x2):
    y = torch.empty_like(x1)
    y[..., 0] = x1[..., 0]*x2[..., 0] - x1[..., 1]*x2[..., 1]
    y[..., 1] = x1[..., 0]*x2[..., 1] + x1[..., 1]*x2[..., 0]
    return y

@curry
def imfilter_real(f_data1, f_data2):
    """
    For convolving f_data1 with f_data2 using PyTorch
    """
    ndim = 2
    f_data1 = torch.from_numpy(f_data1).double()
    f_data2 = torch.from_numpy(f_data2).double()
    rfft = torch_rfft(signal_ndim=ndim)
    irfft = torch_irfft(signal_ndim=ndim)
    return pipe(f_data1,
                rfft,
                lambda x: mult(x, conjugate(rfft(f_data2))),
                irfft,
                fftshift)

@curry
def imfilter(x_data1, x_data2):
    """
    For convolving f_data1 with f_data2 using PyTorch
    """
    ndim = 2
    f_data1 = np.zeros(list(x_data1.shape)+[2,])
    f_data2 = np.zeros(list(x_data2.shape)+[2,])
    f_data1[...,0] = x_data1
    f_data2[...,0] = x_data2
    f_data1 = torch.from_numpy(f_data1).double()
    f_data2 = torch.from_numpy(f_data2).double()
    fft = curry(torch.fft)(signal_ndim=ndim)
    ifft = curry(torch.ifft)(signal_ndim=ndim)
    return pipe(f_data1,
                fft,
                lambda x: mult(x, conjugate(fft(f_data2))),
                ifft,
                lambda x: x[...,0],
                fftshift)

def ch_run_torch(x_data, args):

    N = x_data.shape[1]
    if not np.all(np.array(x_data.shape[1:]) == N):
        raise RuntimeError("x_data must represent a square domain")

    L = args.dx * N
    k = np.arange(N)

    if N % 2 == 0:
        N1 = N // 2
        N2 = N1
    else:
        N1 = (N - 1) // 2
        N2 = N1 + 1

    k[N2:] = (k - N1)[:N1]
    k = k * 2 * np.pi / L

    i_ = np.indices(x_data.shape[1:])

    ksq = np.zeros((1, N, N, 2))

    ksq[...,0] = np.sum(k[i_] ** 2, axis=0)[None]
    ksq[...,1] = np.sum(k[i_] ** 2, axis=0)[None]

    ksq = torch.from_numpy(ksq).double().to(args.device)

    a1 = 3.
    a2 = 0.

    explicit = ksq * (a1 - args.gamma * a2 * ksq)
    implicit = ksq * ((1 - a1) - args.gamma * (1 - a2) * ksq)

    ndim = 2
    fft  = curry(torch.fft)(signal_ndim=ndim)
    ifft = curry(torch.ifft)(signal_ndim=ndim)

    response = torch.zeros(list(x_data.shape)+[2,], dtype=torch.double, device=args.device)
    response[...,0] = x_data
    Fy = fft(response)

    for i in range(args.sim_steps):
        FX  = Fy
        FX3 = fft(response ** 3)
        Fy  = (FX * (1 + args.dt * explicit) - ksq * args.dt * FX3) / (1 - args.dt * implicit)
        response = ifft(Fy)
#         del FX, FX3
#         torch.cuda.empty_cache()
    
    del FX, FX3, Fy, ksq, explicit, implicit
    torch.cuda.empty_cache()
    return response[...,0]

In [1]:
def init_unif(nsamples, dim_x, dim_y,scale=0.1, seed=354875):
    np.random.seed(seed)
    return (2 * np.random.random((nsamples, dim_x, dim_y)) - 1) * scale


def init_norm(nsamples, dim_x, dim_y, scale_mean=0.1, scale_std=0.1, seed=354875):
    np.random.seed(seed)
    means  = (2 * np.random.random(nsamples) - 1) * scale_mean
    x_data = [np.random.normal(loc=m, scale=scale_std, size = (1, dim_x, dim_y)) for m in means]
    x_data = np.concatenate(x_data, axis=0)
    return x_data

def get_fname(output_folder, indx, args):
    return output_folder + "/ch_%d_gamma_%d_dt_%d_dx_%d_%d_%s.npy" % (indx*args.sim_steps, 
                                                                      int(args.gamma*1000), 
                                                                      int(args.dt*1000), 
                                                                      int(args.dx*1000), 
                                                                      args.dim_x, args.init)

def generate(args, n_runs=100, output_folder="indata"):

    if args.init == "unif":
        x_data = init_unif(args.nsamples, args.dim_x, args.dim_y, seed=354875)
    elif args.init == "norm":
        x_data = init_norm(args.nsamples, args.dim_x, args.dim_y, seed=784361)
    
    
    
    y_data = torch.from_numpy(x_data).double().to(args.device)

    np.save(get_fname(output_folder, 0, args), x_data)
    
    print("Starting Execution")
    for i in tqdm.tqdm_notebook(range(n_runs)):
        
        y_data = ch_run_torch(y_data,args)
        fname = get_fname(output_folder, i+1, args)
        np.save(fname, y_data.cpu().numpy())
        
    print("Ending Execution")

In [6]:
class args:
    nsamples=4096
    dim_x=101
    dim_y=dim_x
    sim_steps=100
    dx=0.25
    dt=0.01
    gamma=1.0
    device=torch.device("cuda:0")
    init="norm"

In [7]:
torch.cuda.empty_cache()
generate(args, output_folder="data/norm", n_runs=150)

Starting Execution



Ending Execution


In [ ]:
class args:
    nsamples=4096
    dim_x=101
    dim_y=dim_x
    sim_steps=100
    dx=0.25
    dt=0.01
    gamma=1.0
    device=torch.device("cuda:0")
    init="unif"
torch.cuda.empty_cache()
generate(args, output_folder="data/unif", n_runs=150)

In [15]:
import matplotlib.pyplot as plt
def draw_im(im, title=None):
    im = np.squeeze(im)
    plt.imshow(im)
    plt.colorbar()
    if title is not None:
        plt.title(title)
    plt.show()

In [21]:
data = np.load("data/unif/ch_10000_gamma_1000_dt_10_dx_250_101_unif.npy")

In [27]:
cpuStats()
memReport()

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
2.7
svmem(total=404005154816, available=392542130176, percent=2.8, used=10063011840, free=390920663040, active=4617043968, inactive=958767104, buffers=40955904, cached=2980524032, shared=224067584, slab=1554169856)
memory GB: 2.914287567138672


/storage/home/hhive1/ashanker9/data/anaconda3/envs/chnet/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:100: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
